<a href="https://colab.research.google.com/github/ampham03/grad-assessment/blob/main/grad_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00


In [8]:
import torch
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset

In [12]:
model_name = "Helsinki-NLP/opus-mt-en-de"  # English to German
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

# load WMT dataset for de-en
de_en_dev = load_dataset("wmt19", "de-en", split='validation')

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
def tokenize_sentence(sentence):
    return tokenizer(sentence, return_tensors="pt", padding=True).input_ids

In [2]:
class Hypothesis:
    def __init__(self, tokens, constraints, score = 0):
      self.tokens = tokens
      self.constraints = constraints
      self.score = score
      self.is_open = True


In [3]:
def init_grid(max_len, num_constraints):
  grid = []
  for i in range(max_len):
    row = []
    for j in range(num_constraints + 1):
        row.append([])
    grid.append(row)
  return grid

In [5]:
# generate new open hypotheses
def generate():
  pass

In [ ]:
# start new constrained hypotheses
def start():
  pass

In [ ]:
# continue unfinished hypotheses
def cont():
  pass

In [6]:
def constrained_beam_search(model, input_ids, constraints, max_len, num_constraints, beam_size):

  """
  model: model
  input: example tokenized sentence
  constraints: constraints
  max_len: max_len of translation
  num_constraints: # of constraints
  beam_size: beam_size
  """
  start_hyp = Hypothesis(input_ids, constraints) # initial hypothesis
  grid = init_grid(max_len, num_constraints) # initialize beams in grid
  grid[0][0] = [start_hyp]

  for t in range(1, max_len):
    for c in range(max(0, (num_constraints + t) - max_len), min(t, num_constraints) + 1):
      n, s, g = [], [], []

      for hyp in grid[t - 1][c]:
        if hyp.is_open:
          # g.append(generate())
          pass

      if c > 0:
        for hyp in grid[t - 1][c - 1]:
          if hyp.is_open:
            # n.append(start())
            pass
          else:
            # s.append(cont())
            pass

      # grid[t][c] =  # k-best scoring hypotheses stay on the beam

  # top_level_hyps = grid[][num_constraints]
  # finished_hyps =
  # best_hyp =

  #return best_hyp


In [ ]:
# compute score using BLEU
def compute_score(prediction, actual):
  pass

In [13]:
de_en_dev[0]

{'translation': {'de': 'München 1856: Vier Karten, die Ihren Blick auf die Stadt verändern',
  'en': 'Munich 1856: Four maps that will change your view of the city'}}

In [18]:
tokenize_sentence(de_en_dev[0]['translation']['en'])

tensor([[ 4260,   268,  3441,    16,  9701, 12126,    35,    73,   866,    83,
           704,     7,     4,   642,     0]])